
Connect to Google Drive for datasets (colab)

In [1]:
#from google.colab import drive
#drive.mount('/content/gdrive')

Install dependencies (colab)

In [2]:
#!pip install simpleitk

Dependencies

In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

import SimpleITK as sitk
reader = sitk.ImageFileReader()
reader.SetImageIO("MetaImageIO")

import numpy as np

import os

import pathlib

from natsort import natsorted

#Set GPU/Cuda Device to run model on
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

np.random.seed(46)

Using cuda device


Dataset Directories <Br>
Comment out directory not in use


In [2]:

#Toy Dataset Slices - Removed Empty Slices - Voxel Resampling [2, 0.6, 0.6] Bspline Interpolation 

train_img_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_train_img_slices")
train_label_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_train_label_slices")

test_img_slice_dir = pathlib.Path(r"spider_toy_dset_slices/dummy_test_img_slices")
test_label_slice_dir= pathlib.Path(r"spider_toy_dset_slices/dummy_test_label_slices")



#Local Dataset Slices - Removed Slices
'''
train_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/train_image_slices")
train_label_slice_dir = pathlib.Path("D:/Spider Slice Directories/train_label_slices")

test_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/test_image_slices")
test_label_slice_dir= pathlib.Path("D:/Spider Slice Directories/test_label_slices")
'''
#Local Dataset Slices - Voxel Resampling [2, 0.6, 0.6] Bspline Interpolation 
'''
train_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/train_image_bspline_slices")
train_label_slice_dir = pathlib.Path("D:/Spider Slice Directories/train_label_bspline_slices")

test_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/test_image_bspline_slices")
test_label_slice_dir= pathlib.Path("D:/Spider Slice Directories/test_label_bspline_slices")
'''


'\ntrain_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/train_image_bspline_slices")\ntrain_label_slice_dir = pathlib.Path("D:/Spider Slice Directories/train_label_bspline_slices")\n\ntest_img_slice_dir = pathlib.Path("D:/Spider Slice Directories/test_image_bspline_slices")\ntest_label_slice_dir= pathlib.Path("D:/Spider Slice Directories/test_label_bspline_slices")\n'

Image Slice Class

In [3]:
from transforms import mri_transforms


#TODO add bool image label for interp 
class Mri_Slice:
    def __init__(self, path):
        mri_mha = sitk.ReadImage(path, imageIO = "MetaImageIO") #explicitly setting ioreader just in case

        #grabbing the image as well for downsampling testing
        #self.mri_mha = mri_mha
       
        #get 2d array from mri slice
        mri_a = np.array(sitk.GetArrayFromImage(mri_mha)) #mri_array
        
        mri_a_float32 = mri_a.astype(dtype = np.float32)
        #TODO: set bounds to [-1000, 2000] https://en.wikipedia.org/wiki/Hounsfield_scale
        self.hu_a = mri_a_float32

Zero crop sanity check <br>
Testing Linspace and value mapping

In [4]:
from transforms import array_transforms

test_image_slice = Mri_Slice(r"spider_toy_dset_slices/dummy_train_img_slices/1_t1_30.mha")
test_label_slice = Mri_Slice(r"spider_toy_dset_slices/dummy_train_label_slices/1_t1_30.mha")

#print(test_label_slice.mri_mha)

test_image_a = test_image_slice.hu_a
test_label_a = test_label_slice.hu_a

test_image_a, test_label_a = array_transforms.crop_zero(test_image_a, test_label_a)

test_image_crop_mha = sitk.GetImageFromArray(test_image_a)

test_label_tensor = torch.from_numpy(test_label_a)
        
test_label_tensor = test_label_tensor.to(torch.int)

val_range = np.arange(20)
print(val_range)
key_range = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 100, 201, 202, 203, 204, 205, 206, 207, 208, 209])

#pad to max resolution of slice in dset 
#label_tensor = tensor_transforms.pad_to_resolution(label_tensor, [row_dim_max, col_dim_max]) #torch.functional.pad takes care of cropping, no need to call array_transforms.crop_zero()

print(np.unique(test_label_a))

mapping_dict = dict(zip(key_range, val_range))

# Define a function to perform the mapping using the dictionary
def map_to_specified_set(value):
    return mapping_dict.get(value, np.nan)  # Return np.nan for values not found in the mapping_dict

# Create a vectorized version of the mapping function
value_map = np.vectorize(map_to_specified_set)

# Apply the mapping function to the array
mapped_array = value_map(test_label_a)

print("test arr after mapping", np.unique(mapped_array))
#sitk.WriteImage(test_image_crop_mha, "D:/sitk dump/1_t1_30_cropzero.mha")

#TODO continue here make images 1:1 scale and downsample to 64x64
#if(sitk_slice.GetSize()[0] != sitk_slice.GetSize()[1])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[  0.   1.   2.   3.   4.   5.   6. 100. 201. 202. 203. 204. 205. 206.]
test arr after mapping [ 0  1  2  3  4  5  6 10 11 12 13 14 15 16]


Sort directories


In [4]:
#get lists from directories

#toy dset slices
image_path = train_img_slice_dir
label_path = train_label_slice_dir

image_dir_list = os.listdir(image_path)
label_dir_list = os.listdir(label_path)

print(image_path) 
print(label_path)

#local dset
'''
image_dir_list = os.listdir(local_img_idr)
label_dir_list = os.listdir(local_label_dir)
'''
#sort lists
image_dir_list = natsorted(image_dir_list)
label_dir_list = natsorted(label_dir_list)
#empty lists to hold x and y dimensions of images
row_list = []
col_list = []

#dirlen = len(os.listdir(dummy_train_label_dir))
dirlen = len(os.listdir(label_path))

print("train images count", dirlen)

#print(local_label_idr)


spider_toy_dset_slices\dummy_train_img_slices
spider_toy_dset_slices\dummy_train_label_slices
train images count 2100


Get max dimension of slice in dset for x y padding <br>
Images have slices with 0 label info removed and are cropped using zero crop <br>
Also get min max values of dset for tensor normalization <br>
<b> Results of runs in cell below this one <b> 

In [ ]:
from transforms import array_transforms

row_list = []
col_list = []

for idx in range(0, dirlen):
  #print("dirlen", dirlen)
  
 #toy dset 
  
  img_path = image_path.joinpath(image_dir_list[idx])
  lbl_path = label_path.joinpath(label_dir_list[idx])#first part before joinpath is pathlib.Path, second part is the directory of the file 
  '''
  img_path = local_img_idr.joinpath(image_dir_list[idx])
  label_path = local_label_dir.joinpath(label_dir_list[idx]) #first part before joinpath is pathlib.Path, second part is the directory of hte file 
  '''
  image = Mri_Slice(img_path)
  label = Mri_Slice(lbl_path)

  image_a = image.hu_a
  label_a = label.hu_a

  #crop zero
  image_a_cropzero, label_a_cropzero = array_transforms.crop_zero(image_a, label_a)

  #find array min max values for normalising in Dataset class
  if(idx ==0):
    
    image_tensor_min = np.min(image_a_cropzero)
    image_tensor_max = np.max(label_a_cropzero)
    label_tensor_min = np.min(label_a_cropzero)
    label_tensor_max = np.max(label_a_cropzero)
    
    unique_masks_a = np.unique(label_a)
  else:
    
    if(np.min(image_a_cropzero) < image_tensor_min):
      image_tensor_min = np.min(image_a_cropzero)
      image_tensor_min_dir = img_path
    if(np.min(label_a_cropzero) < label_tensor_min):
      label_tensor_min = np.min(label_a_cropzero)
    if(np.max(image_a_cropzero) > image_tensor_max):
      image_tensor_max = np.max(image_a_cropzero)
    if(np.max(label_a_cropzero) > label_tensor_max):
      label_tensor_max = np.max(label_a_cropzero)
    
  #find amount of unique masks for one-hot encoding 
    current_masks_a = np.unique(label_a)
    if(len(current_masks_a) > len(unique_masks_a)):
      unique_masks_a = current_masks_a
  #print("image res", image_a_cropzero.shape)
  
  
  row_list.append(image_a_cropzero.shape[0]) #add row value to list
  #print(image_a_cropzero.shape[0])
  col_list.append(image_a_cropzero.shape[1]) #add col value to list 
  
  

#calculate max 

row_dim_max = max(row_list)
col_dim_max = max(col_list)

row_dim_max = ((row_dim_max + 15) // 16) * 16 #nearest multiple of 16 
col_dim_max = ((col_dim_max + 15) // 16) * 16 #nearest multiple of 16

print("row max:", max(row_list))
print("col max:", max(col_list))

print("image tensor min", image_tensor_min)
print("image tensor max", image_tensor_max)
print("label tensor min", label_tensor_min)
print("label tensor max", label_tensor_max)

print("amount of masks", len(unique_masks_a))
print("masks array", unique_masks_a)





Results from running above cell to save time

In [5]:
#Toy Dset Bspline Resample 
row_dim_max = 448
col_dim_max = 224

image_tensor_min = -2057.0
image_tensor_max = 4292.0
label_tensor_min = 0.0
label_tensor_max = 208.0 #IVD 209 IS NOT EXISTENT NO CLUE WHY 

'''
#Local Dset not resampled
row_dim_max = 928
col_dim_max = 400

image_tensor_min = -1000
image_tensor_max = 3096
label_tensor_min = 0.0
label_tensor_max = 209.0
'''

#Local dset bspline resample
'''
row_dim_max = 464
col_dim_max = 224

image_tensor_min = -2230.0
image_tensor_max = 4385.0
label_tensor_min = 0.0
label_tensor_max = 209.0
'''
unique_masks_a = np.array([0. ,   1. ,  2.   ,3.  , 4. ,  5. ,  6. ,  7. ,  8.,   9. ,100. ,201. ,202. ,203. ,204., 205. ,206. ,207. ,208. ,209.])


Value mapping for one-hot encoding label tensors (needed for BCE to work)

In [6]:
val_range = np.arange(20)
print(val_range)
key_range = unique_masks_a #no of unique values in labels, including BG 
print(key_range)

mapping_dict = dict(zip(key_range, val_range))

# Define a function to perform the mapping using the dictionary
def map_to_specified_set(value):
    return mapping_dict.get(value, np.nan)  # Return np.nan for values not found in the mapping_dict

# Create a vectorized version of the mapping function
value_map = np.vectorize(map_to_specified_set)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9. 100. 201. 202. 203.
 204. 205. 206. 207. 208. 209.]


Dataset Class

In [7]:
from transforms import tensor_transforms

class SpiderDataset(Dataset):
    def __init__(self, labels_dir, img_dir, transform=None, target_transform=None):
        self.labels_dir = labels_dir
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(os.listdir(self.labels_dir))

    def __getitem__(self, idx):
        label_dir_list = os.listdir(self.labels_dir)
        image_dir_list = os.listdir(self.img_dir)

        image_dir_list = natsorted(image_dir_list)
        label_dir_list = natsorted(label_dir_list)

        img_path = self.img_dir.joinpath(image_dir_list[idx])
        label_path = self.labels_dir.joinpath(label_dir_list[idx])

        image = Mri_Slice(img_path)
        label = Mri_Slice(label_path)

        image_a = image.hu_a
        label_a = label.hu_a

        #value mapping label tensor 0-19
        label_a = value_map(label_a)

        image_tensor = torch.from_numpy(image_a)
        label_tensor = torch.from_numpy(label_a)
        
        image_tensor = image_tensor.to(torch.float32)
        label_tensor = label_tensor.to(torch.float32) #techincally these are ints not floats 

        #pad to max resolution of slice in dset 
        image_tensor = tensor_transforms.pad_to_resolution(image_tensor, [row_dim_max, col_dim_max])
        label_tensor = tensor_transforms.pad_to_resolution(label_tensor, [row_dim_max, col_dim_max]) #torch.functional.pad takes care of cropping, no need to call array_transforms.crop_zero()

        #normalise image tensor
        image_tensor = (image_tensor - image_tensor_min) / (image_tensor_max - image_tensor_min)
        #label_tensor = (label_tensor - label_tensor_min) / (label_tensor_max - label_tensor_min) #INTS HERE DO NOT NORMALISE 

        #one hot encoding label tensor BUG FIX
        #one hot label, this works only if the range on the label tensors is from 0 to 19 steps of 1 
        label_tensor = nn.functional.one_hot(label_tensor.long(), num_classes= 20) #resulting shape is torch.Size([8, 20, 448, 224]

        
        

        label_tensor = label_tensor.float()

        #print("tensor dims", image_tensor.shape)
        #print("label tensor min clamp",torch.min(label_tensor))
        #print("label tensor max clamp", torch.max(label_tensor))

        

        image_tensor = image_tensor.unsqueeze(0)
        #label_tensor = label_tensor.unsqueeze(0) 
        
        #print("label before permute shape", label_tensor.shape)
        #permute axes label tensor
        label_tensor = torch.permute(label_tensor, (2, 0, 1))

        #remove backround (0) channel in channel dim of label tensor 
        
        label_tensor = label_tensor[1:, :, :] #TODO: current iteration removes some vertebra and trains model on backround fix 
        
        #print("tensor shape after removing first channel 2nd dim", label_tensor.shape)

        label_flattened = label_tensor.flatten()
        print("label tensor unique values", torch.unique(label_flattened))

        image_tensor = image_tensor.to(device)
        label_tensor = label_tensor.to(device)

        #print(image_tensor.shape)
        return image_tensor, label_tensor




Dataset Class Instances


In [8]:
#toy train test dataset to test network running
#local_train_set = SpiderDataset(local_img_idr, local_label_idr)
dummy_train_set = SpiderDataset(train_label_slice_dir, train_img_slice_dir)

dummy_test_set = SpiderDataset(test_label_slice_dir, test_img_slice_dir)

print("train dataset len",dummy_train_set.__len__())
print("test dataset len",dummy_test_set.__len__())

train dataset len 2100
test dataset len 215


Create Unet Instance







In [9]:
from models import unet 

input_channels = 1 #Hounsfield scale
output_channels = 19 #one for every class 0-9 vertebrae 10 spinal canal 11-19 ivd
start_filts = 32 #unet filters 
up_mode = 'upsample' #options are either 'upsample' for NN upsampling or 'transpose' for transpose conv

model = unet.UNet(in_channels= input_channels,num_classes=output_channels, start_filts=start_filts, up_mode=up_mode) #testing model hyperparams
model.to(device)
model.to(torch.float32)
#for param in model.parameters():
 #   print(param.device)

d:\Github Repos\Spider Seg e19005\spider-seg-e19005\models\unet.py:213: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(m.weight)
d:\Github Repos\Spider Seg e19005\spider-seg-e19005\models\unet.py:214: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias, 0)


UNet(
  (conv_final): Conv2d(32, 19, kernel_size=(1, 1), stride=(1, 1))
  (down_convs): ModuleList(
    (0): DownConv(
      (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): DownConv(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): DownConv(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (3): DownConv(
      (conv1): Conv2d(128, 256, kernel_size=(3, 3),

Create Faster-RCNN Instance trained


In [32]:
'''
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes

weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
'''

'\nfrom torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights\nfrom torchvision.utils import draw_bounding_boxes\n\nweights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT\nmodel = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)\n'

Create Mask RCNN Instance Trained


In [12]:
'''
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN_ResNet50_FPN_V2_Weights

model = maskrcnn_resnet50_fpn_v2(
    weights = MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT,
    num_classes = 19
    #weights_backbone = 
)
'''

'\nfrom torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN_ResNet50_FPN_V2_Weights\n\nmodel = maskrcnn_resnet50_fpn_v2(\n    weights = MaskRCNN_ResNet50_FPN_V2_Weights.DEFAULT,\n    num_classes = 19\n    #weights_backbone = \n)\n'

Hyperparameters


In [10]:
epochs = 15 #hyperparam experiments
lr = 0.001 
batchsize = 8 #max on local machine
loss_func = nn.BCEWithLogitsLoss() #regular bce isn't doing that well 
loss_func.to(device)
optim = torch.optim.Adam(model.parameters(), lr=lr)

print(epochs)
print(lr)
print(batchsize)
print(loss_func)
print(start_filts)
print(up_mode)

print((loss_func))

15
0.001
8
BCEWithLogitsLoss()
32
upsample
BCEWithLogitsLoss()


Dataloader

In [11]:
dummy_train_dataloader = DataLoader(dummy_train_set, batch_size = batchsize, shuffle=True)

dummy_test_dataloader = DataLoader(dummy_test_set, batch_size = batchsize, shuffle=True)


'''
for batch in dummy_train_dataloader:
    for tensor in batch:
        print("min", torch.min(tensor))
        print("max", torch.max(tensor))
'''

'\nfor batch in dummy_train_dataloader:\n    for tensor in batch:\n        print("min", torch.min(tensor))\n        print("max", torch.max(tensor))\n'

Tensor Dimensions Check

In [13]:
'''
for idx_slice in dummy_train_dataloader:
    for tensor in idx_slice:
        print(tensor.shape)
        break
'''

'\nfor idx_slice in dummy_train_dataloader:\n    for tensor in idx_slice:\n        print(tensor.shape)\n        break\n'

One Epoch <br>
https://pytorch.org/tutorials/beginner/introyt/trainingyt.html

In [12]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting

    #swap train dataloader for dset
    for i, data in enumerate(dummy_train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        #print("labels", labels.shape)
        #print(inputs.shape)
        #print(device)

        #inputs = inputs.transpose(-1,0)
        #labels = labels.transpose(-1,0)
        #inputs = inputs.reshape(inputs.shape(1), inputs.shape(0), inputs.shape(3), inputs.shape(4))
        #labels = labels.reshape(labels.shape(1), labels.shape(0), labels.shape(3), labels.shape(4))

      
        # Zero your gradients for every batch!
        optim.zero_grad()

        #2d
        # Make predictions for this batch
        outputs = model(inputs)

        '''
        print("inputs", inputs.shape)
        print("outputs", outputs.shape)
        print("labels", labels.shape)
        '''
        
        # Compute the loss and its gradients
        loss = loss_func(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optim.step()

        #3d tensor, probably won't use keeping here just in case  
        '''
        #run everything while indexing through z axis
        for axis in images, masks:
          for idx in range(0, axis.size(1)):
            # Zero your gradients for every batch!
            optim.zero_grad()
            print(inputs[:, idx, : ,:])
            # Make predictions for this batch
            outputs = model(inputs[:, idx, : ,:])

            # Compute the loss and its gradients
            loss = loss_func(outputs[:, idx, : ,:], labels[:, idx, : ,:])
            loss.backward()

            # Adjust learning weights
            optim.step()
        '''
        # Gather data and report
        running_loss += loss.item()
        #if i % 1000 == 999:
            #print("goes in")
        last_loss = running_loss / 1000 # loss per batch
        print('  batch {} loss: {}'.format(i + 1, last_loss))
        tb_x = epoch_index * len(dummy_train_dataloader) + i + 1
        tb_writer.add_scalar('Loss/train', last_loss, tb_x)
        running_loss = 0.
        #if ends here

    print("loss", loss)
    return last_loss

Train Loop <br>
https://pytorch.org/tutorials/beginner/introyt/trainingyt.html


In [ ]:
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/spider_seg_unet_epochs={}_lr={}_batchsize={}_loss=BCEWithLogits_startfilts={}_upmode={}'.format(epochs,lr, batchsize,start_filts,up_mode))
epoch_number = 0


best_vloss = 1_000_000.

for epoch in range(epochs):
    print('EPOCH {}:'.format(epoch_number + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)
    #print("avg loss in epoch", avg_loss)

    running_vloss = 0.0
    # Set the model to evaluation mode, disabling dropout and using population
    # statistics for batch normalization.
    model.eval()

    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(dummy_test_dataloader):
            vinputs, vlabels = vdata

            voutputs = model(vinputs)
            vloss = loss_func(voutputs, vlabels)
            running_vloss += vloss
            

    avg_vloss = running_vloss / (i + 1)
    #print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and validation
    
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    
    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
    
        
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)
    
        
    epoch_number += 1